In [ ]:
import os

from tqdm import tqdm

from lmsiage.uncertainty import ComputeAgeUncertainty
from lmsiage.zarr_index_tools import cleanup_missing_files, files_with_arrays, update_index_for_dir, files_missing_array


In [ ]:
force = False
sia_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr'
mesh_dir = f'{sia_dir}/mesh'
age_dir = f'{sia_dir}/age'
unc_dir = f'{sia_dir}/uncert'

In [ ]:
cleanup_missing_files()
update_index_for_dir(unc_dir)

In [ ]:
ready_unc_files = files_with_arrays(['unc_sic', 'unc_sid'])
ready_unc_files = [f for f in ready_unc_files if os.path.basename(f).split('_')[1].split('.')[0] > '19910914']

if force:
    unproc_unc_files = ready_unc_files
else:
    unproc_unc_files = files_missing_array('unc_age')
    unproc_unc_files = sorted(list(set(ready_unc_files).intersection(set(unproc_unc_files))))

print(unproc_unc_files[0], unproc_unc_files[-1], len(unproc_unc_files))    

In [ ]:
compute_age_unc = ComputeAgeUncertainty(mesh_dir, age_dir, unc_dir, force=force)
for unc_file in tqdm(unproc_unc_files):
    status = compute_age_unc(unc_file)
    if status == -1:
        print(f'No uncertainty years found in {unc_file}, skipping')
    if status == -2:
        print(f'Uncertainty already computed in {unc_file}, skipping')


In [ ]:
from lmsiage.mesh_file import MeshFile
import matplotlib.pyplot as plt

year = 2000
mmdd = '1231'
unc_file = f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/{year}/unc_{year}{mmdd}.zip'
age_file = f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/{year}/unc_{year}{mmdd}.zip'
mesh_file = f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/{year}/mesh_{year}{mmdd}.zip'

print(MeshFile(unc_file).read_names())
unc_age, unc_fracs = MeshFile(unc_file).load(['unc_age', 'unc_fracs'], as_dict=False)
x, y, t = MeshFile(mesh_file).load(['x', 'y', 't'], as_dict=False)

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
axs[0].tripcolor(x, y, t, unc_age, clim=[0, 2])
axs[1].tripcolor(x, y, t, unc_fracs[0], clim=[0, 20])
for ax in axs:
    ax.set_aspect('equal')
    plt.colorbar(ax.collections[0], ax=ax)
plt.show()

In [ ]:
3280 + 264